In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Import library

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Read data

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')

x_data = train.drop(['id', 'claim'], axis=1)
x_test = test.drop(['id'], axis=1)

y_data = train.claim

### Split train and val data

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data)

### Train model

In [ ]:
model = XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor')
model.fit(x_train, y_train)
y_pred = model.predict_proba(x_val)[:,1]

acc = roc_auc_score(y_val, y_pred)
print(f'validation roc auc score {acc}')
print(f'train roc auc score {roc_auc_score(y_train, model.predict_proba(x_train)[:, 1])}')

### Predict test label

In [ ]:
y_test = model.predict_proba(x_test)[:, 1]

### Save prediction and submission

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')
submission.claim = y_test
submission.to_csv('submission.csv', index=False)